# Universal differential equations

Esse notebook é uma demonstração de 'Universal differential equations' (UDE) usando modelos de circuitos equivalentes e PyBaMM. Uma UDE é uma equação diferencial definito, por todo ou em parte, em função de "aproximadores universais" (redes-neurais nesse caso). 

Na prática, são sistema físicos, descritos por equações diferenciais, onde um dos componentes é uma rede neural[1]. Exemplo: $$\frac{\partial u}{\partial t} = f(x,u,t) + \mathcal{NN}(x,u)$$

Vantagem dessa implementação: $f(\cdot)$ pode ser um modelo físico simplificado (*reduced order*) que retorna uma mera aproximação do resultado real. Nesse caso, a rede neural $\mathcal{NN}$ pode ser menor, mais simples, responsável somente por corrigir o erro, a divergência.

---
1. Rackauckas et al 2021 (https://doi.org/10.48550/arXiv.2001.04385).

## Exemplo Neural-TECMD
O melhor exemplo, com código open-source disponível online, vem de Kuzhiyil et al, 2024 [2, 3]. O software é implementado em Julia, consegui faze-lo rodar localmente, mas não no Google Colab. Vou demonstrar abaixo a lógica básica do programa e o output do código -- não será possível executar localmente agora.

O artigo descreve o processo de combinar modelo  *Thermal Equivalent Circuit Model with Diffusion* (TECMD) -- um dos modelos do tipo ECM mais sofisticados -- com duas redes neurais que corrigem o erro de temperatura e tensão. Na figura abaixo, blocos pretos representam modelos físicos-matemáticos, blocos laranjas representam redes neurais.

![test](imgs/neural_tecmd_kuzhiyil.jpg)

O modelo conseguiu obter reduções de erro significativo, enquanto preserva performance.

A figura abaixo demonstra a comparação entre dados experimentais, e os resultados da simulação com e sem redes-neurais auxiliares.

![test](imgs/neural_tecmd_plot.png)

---
2. Kuzhiyil et al 2024 (https://doi.org/10.1016/j.apenergy.2024.123692)
3. https://github.com/JishnuKuzhiyil/Neural-TECMD-Battery-model

## Usando PyBaMM

Python Battery Mathematical Modelling (PyBaMM) é um pacote de Python open-source para simulação eletroquímica de baterias.

In [ ]:
# importing basic libs
import os
import sys

# setting up code to run in google colab
git_repo_url = "https://github.com/SarvTex/june_sciml_workshop.git"
repo_name = "june_sciml_workshop"

if 'google.colab' in sys.modules:
    # Google colab setup
    print("> Code running in Colab env ===")

    if not repo_name in os.getcwd().split(os.sep) and not repo_name in os.listdir():
        print("> Cloning repo (dataset) ===")
        !git clone {git_repo_url}
    else:
      print("> Repo already cloned ===")

    if repo_name in os.listdir():
        os.chdir(repo_name)
    print(f"> Currently in correct folder ({os.getcwd()}) ===")

else:
    # Running locally setup
    print("> Running locally. Assuming correct folder structure ===")

!ls